In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

import torch
import pytorch_lightning as pl


import dataset
import neural_network
import config as cfg

## Parameters

In [ ]:
CLASSES = dataset.metadata.get_classes(cfg.paths.LABELS_CSV["train"])  
CLASSES, CLASSES.size

In [ ]:
IMG_CHANNELS, IMG_HEIGHT, IMG_WIDTH = dataset.metadata.get_image_dimensions(cfg.paths.IMG_DIR["train"])
IMG_CHANNELS, IMG_HEIGHT, IMG_WIDTH

## Loading Checkpoint

In [ ]:
SEED = 0
pl.seed_everything(SEED, workers=True)

In [ ]:
data_module = dataset.SkinCancerDataModule(
    cfg.paths.LABELS_CSV,
    cfg.paths.IMG_DIR,
    cfg.hparams.BATCH_SIZE,
    cfg.hparams.DATALOADER_NUM_WORKERS,
    transform=None
)

In [ ]:
model = neural_network.ConvNetwork(CLASSES.size, cfg.hparams.DROPOUT_RATE)
print(model)

In [ ]:
version = 0
CHECKPOINT_DIR = cfg.paths.LOG_DIR / f"lightning_logs/version_{version}" / "checkpoints"

CHECKPOINT_PATHS = list(CHECKPOINT_DIR.glob("*.ckpt")) 
CHECKPOINT_PATHS

In [ ]:
model_module: neural_network.NetworkModule = neural_network.NetworkModule.load_from_checkpoint(
    CHECKPOINT_PATHS[0],
    model=model,
    channels=IMG_CHANNELS,
    height=IMG_HEIGHT,
    width=IMG_WIDTH,
    num_classes=CLASSES.size,
    learning_rate=cfg.hparams.LEARNING_RATE
)

## Prediction

In [ ]:
from pytorch_lightning.loggers import CSVLogger

logger = CSVLogger(cfg.paths.LOG_DIR)

In [ ]:
trainer = pl.Trainer(logger=logger, deterministic=True)

model_module.clear_test_predictions_variables()
trainer.test(model_module, data_module)

In [ ]:
true_labels_pos = torch.concat(model_module.test_expected).cpu()

predicted_probabilities = torch.concat(model_module.test_probabilities)
predicted_labels_pos = predicted_probabilities.argmax(dim=1).cpu()

true_labels = CLASSES[true_labels_pos]
predicted_labels = CLASSES[predicted_labels_pos]

## ROC

In [ ]:
fpr, tpr, roc_auc = {}, {}, {}

for i in range(CLASSES.size):

    expected = (true_labels_pos == i).type(torch.int64)  # one for i-th class, zero for the others
    probabilities = predicted_probabilities[:, i] # probabilities for prediction i-th class

    fpr[i], tpr[i], _ = roc_curve(expected, probabilities)
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
# MEAN TVP OVER ALL CLASSES
all_classes_fpr = np.unique([value for i in range(CLASSES.size) for value in fpr[i]])  # x-axis

media_tvp = np.zeros_like(all_classes_fpr)

for i in range(CLASSES.size):
    media_tvp += np.interp(all_classes_fpr, fpr[i], tpr[i])

media_tvp /= all_classes_fpr

fpr["macro"] = all_classes_fpr
tpr["macro"] = media_tvp
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

In [ ]:
plt.figure(figsize=(8,6))

LABELS_MACRO = f"Media macro curva ROC (AUC = {roc_auc['macro']:.2f})"

plt.plot(fpr["macro"], tpr["macro"], label=LABELS_MACRO, linestyle=":", linewidth=4)

LABEL_CLASSES = {
    i: f"Curva ROC da classe {class_} (AUC = {roc_auc[i]:.2f})" 
    for i, class_ in enumerate(CLASSES)
}

for i, label in LABEL_CLASSES.items():
    plt.plot(fpr[i], tpr[i], lw=2, label=label)

plt.plot([0, 1], [0, 1], "k--",color="red", lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.annotate("Decisão aleatória",(.5,.48),color="red")
plt.xlabel("Taxa de Falsos Positivos")
plt.ylabel("Taxa de Verdadeiros Positivos")
plt.title("Curva ROC")
plt.legend(loc="best")
plt.savefig("auc_roc.png")
plt.show()